## Data Science Immesive Final Project (General Assembly ATX Spring 2017)
The data for my final project was obtained from the Appalachian Trail Conservancy (Harpers Ferry,WV).

### Final Project Question: Can you predict how long it will take a thru hike to complete the Appalachian Trail based on the features of this dataset?



# Data Cleaning Table of Content: 
I explore the data column by column to understand what sort of missing or inaccurate values are contained.

def (clean_1): Columns that can/will be dropped
- Rename columns to be more accurate.
- Drop 'Days Elapsed' column as it does not accuratly calculate the number of days on trail. (I replace this column in clean_2 with 'Total_Days')
- Drop 'Hiking_Disability' Column as it was not well reported
- Drop Dog Weight Column (16238 of possible 17835 values were 0)
- Drop columns with too many null values (columns that exceed 85 percent null values)
- drop null values out of columns
- **** come back to this Drop Under2kAmount column since (16125 out of 17835) values were 0 essentially the same as null values 
- Dropped 'Hiking 


def (clean_2/clean_3): Clean up the Age column 
- Fill null values in 'Age' column with median age.
- All values below 4(There were some negative values) 
- The youngest person to thru-hike the AT is 5 so I assume all values above this are 

 
    
def (clean_4): Datetime objects, 
-  Convert 'Start' and 'Finish' columns to Datetime objects
-  Create a new column 'Total Days' that calculates total days on trail (at['Finish] - at['Start'])) 
-  There was as single null value in completion year, however, the finish date showed the hiker completed in 2016 so I changed the value to 2016 
- I conveted the completion yr column from floats to ints 
- converted completion yr to a datetime object



def (clean_5/clean_6): Clean 'Sex' column
- In (clean_1) I dropped all null values in this column 
- Drop 'oher' and 'othe' out of the sex column
- Convert the 'Sex' column to numerical format with the map function
- Drop null values in the Sex column. There were only 2. 
- Converted M/F from float to int

def (clean_7/clean_8): Clean 'Hike_Type' column (Drop section, alternative hikes, Yo Yo)
- Drop values in the Hike column that are labeled alternative as I am unsure what these are and how to define them.
- Since I am only interested in Thru Hikers and trying to predict time on trail based on people trying to hike in 6 months, and since all thru section hikers take varying amounts of time. 
- Difficult to estimate value for a Yo Yo so I dropped it, it was only a single column

def (clean_9): Categorize the race column: Since the Data Set was very disorganized, Regular Expression
was unable to take care of the categorization so I had to manually manipulate the categories
- Caucasion variation = 0
- African American Variation = 1
- Asian Variation = 2
- Hispanic Variation = 3
- Native American = 4
- Mixed Race: =5
- European = 6
- Mixed Race  = 7
- European/Not American (You cannot assume they are white because they are European) = 8
- Other = 9
    

def(clean_10): Group country of origin to express if they are North American, European, Australian,etc...
- North American = 0 
- European = 1 
- Oceania = 2
- African = 3

def(clean_11): Convert Hike_Type values to numbers
- 'NOBO':0
- 'SOBO':1 
- 'Flip':2 

def (clean_12): Reset Index 
- The index needed to be reset since I had dropped many rows containing null values 



# To do 

3.) Make Boolean columns into dummy variables 

7.) State 

8.) There are negative total days (str.replace values)




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
at = pd.read_excel('BenHikersInfo.xlsx')

## Cleaning Dataset

In [3]:
def clean_1(df):
    df.drop(labels = df[['Disease Other','School','Hiking Disability','DogWeightLbs','Days Elapsed','MinorityComments','School',
                         'DogBreed','Disability','Under2KAmount']], axis = 1, inplace = True)
    df.rename(columns = {'AgeYrFin': 'Age','Race or Heritage': 'Race',
                        'StAbbrev':'State', 'Hiking Disability ': 'Hiking_Disability', 
                         'Hike Type': 'Hike_Type', 'Yr Comp':'completion_yr',
                         },inplace = True)
    df.dropna(subset= ['Start','Sex','Race','Country','State','Hike_Type'],inplace = True)

clean_1(at)  


def clean_2(df):
    df['Age'] = df['Age'].fillna(df['Age'].median())
clean_2(at)


def clean_3(x,median_age): 
    if x < 4:
        return median_age
    else:
        return x 
at['Age'] = at['Age'].apply(lambda x: clean_3(x, at['Age'].median()))

In [4]:
def clean_4(df):
    df['Finish'] = pd.to_datetime(df['Finish'])
    df['Start'] = pd.to_datetime(df['Start'])
    df['Total_Days'] = df['Finish'] - df['Start']
    df['completion_yr'].fillna(2016,inplace = True)
    df['completion_yr'] = df['completion_yr'].astype(int)
clean_4(at)

In [5]:
def clean_5(df):
    df['Sex'] = df['Sex'].map({'M':0,'F':1})
    to_drop = ['othe','Oher']
    x = df[df['Sex'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_5(at)

def clean_6(df):
    df.dropna(subset = ['Sex'], inplace = True)
    df['Sex'] = df['Sex'].astype(int)
clean_6(at)

In [6]:
def clean_7(df):
    to_drop = ['Alternative','alternative','Alterntive', 'Yo Yo']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_7(at)

def clean_8(df):
    to_drop = ['Section']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_8(at)

In [7]:
# Recall to maintain original column 
def clean_9(df):
    
    # Caucasion Variation: 0 Identify Race Over Nationality/Religion 
    df['Race'] = df['Race'].replace(['White','WASP','Hewbrew/White','English','Caucasian / Viking',
                                    'Caucasian or White','Caucasian or White/Irish','Caucasian / Polish / Irish',
                                    'Caucasian / South Africaan','Caucasian or White/Amish','Caucasian / Irish'
                                     'Caucasian or White/Jewish','Caucasian / Irish',
                                     'Anglo Saxon / Scottish / American','Caucasian or White (Jewish)',
                                     'Caucasian / European','Caucasian or White/Pacific Islander','Jewish/White',
                                     'Caucasian or White/Jewish','Caucasian / non-Hispanic',
                                     'Caucasian / Czech / Russian / German / English)','Anglo Saxon','White/Irish',
                                     'White/Jewish','Caucasian/Lithuanian-American','White/Scandinavian','Caucasian',
                                     'White/Pennsylvania Dutch','Caucasian / Irish / Nordic','Caucasian / Jewish',
                                     'European/American', 'Caucasian / Polish','Caucasian / German',
                                     'Caucasian / 1/4 German', 'Caucasian/French Canandien','Caucasian / Slovak','Mennowhite',
                                     'White/Lebanese', 'White - Irish, Scot and Spanish', 'Caucasian / Italian / Eastern European',
                                     'Caucasian / Bahamian', 'Caucasian / Haitian'], 0)
    # African American Variation: 1
    df['Race'] = df['Race'].replace(['African American','African American/Biracial',], 1)
    
    # Asian Variation: 2
    df['Race'] = df['Race'].replace(['Asian','Chinese','Asian / Chinese','Vietnamese','Thai',
                                    'Taiwanese / American','Asian / American','Korean/American','Japanese',
                                    'Indian','Asian American','Korean / American','Asian (Indian/American)',
                                    'Middle Eastern','Korean','Japanese/American','Caucasian / Asian / Pacific Islander',
                                     'American Indian', 'Vietnamese American','Persian','Japanese / American', 'Pakistani',
                                     'Tamil/Sri Lankan', 'Pacific Islander', 'Caucasian / Asian'], 2)
    # Hispanic/Latino: 3 I prioritize race over nationality 
    df['Race'] = df['Race'].replace(['Hispanic','Latino / Venzuelan','Japanese-American','Mexican / American',
                                    'Mexican','Hispanic  / Colombian','Cuban','Latino','Puerto Rican',
                                     'Hispanic or Latino','Mexican/American','Dominican','Hispanic / Mexican',
                                    'Brazilian','Hispanic / Mexican / American','Hispanic/German', 'Hispanic / Italian',
                                    ], 3)
    # Native American Variation: 4
    df['Race'] = df['Race'].replace(['American Indian / Cherokee','Cherokee & English','Native American',
                                    'Native American (1/2 cherokee) / 1/2 Celtic','American Indian / Choctaw'],4)
    
    # Mixed Race: 5 (remember to look at this in relation to all the mixed white people/lation peop)
    df['Race'] = df['Race'].replace(['Mixed','mixed','Caucasian / Black / West Indian','Hispanic/Black','Black/White',
                                    'Mix','Black and White','white/hispanic','White/Hispanic',
                                     'Mutt','Black / Caucasian','European Mutt','White/Asian',
                                     'White & African American','Caucasian / Chinese', 'Caucasian / Philipino',
                                     'Hispanic or Latino; Mixed','Japanese/White','White/Latina','Hispanic/White',
                                     'Latino/White', 'Caucasian/Hispanic','Hispanic / Caucasian','Caucasian/Pacific Islander',
                                     'Filipino/Caucasian', 'African American/American Indian','White/Native American',
                                     'Middle Easter/White', 'European/Native American', 'Caucasian / Brazilian',
                                     'Asian / Pacific Islander / Hispanic', 'Caucasian / Hispanic', 'Caucasion/Filipino',
                                     'American Indian / Caucasian', 'Lebanese / American','Caucasian / Pacific Islander',
                                     'Navajo / Portugese','WASP / Italian'], 5)
    
    # European: 6 (You cannot assume they are white because they are European)
    df['Race'] = df['Race'].replace(['German','Italian','Polish','Dutch','European','Swiss','North European',
                                    'Canadian','Finnish','german','Scottish','Ukrainian',
                                    'Romanian','Bulgarian','Spanish','Lithuanian','British or English','Scandinavian',
                                     'Western European','Irish','Scot - Irish','French-Canadian','Swiss/American',
                                     'Eastern European', 'French Acadian'], 6)
    
    # Race Not on Application/ Not specified: 7 
    df['Race'] = df['Race'].replace(['Prefer not to say', 'No Application - Letter Only','Not on Application', 'not latino or hispanic',
                                     'not hispanic or lation','Not hispanic or Lation','not hispanice or latino',
                                     'Not Hispnic or Latino', 'Not hispanic or Latino', 'Not Hispanice or Latino',
                                     'Not Hispanic of Latino', 'Not Hispanic or Latino','Not hispanic or latino',
                                     'nor hispanic or latino','not hispanic or latino','Not Hispanic or Lation',
                                     'Not Hispanic or latino'], 7)
    
    # Mixed Nationality 8
    df['Race'] = df['Race'].replace(['German/Irish American','Irish/American','Italian American','German / Slovak',
                                     'Yugoslav / Swedish','Irish / Scottish / German', 'Chinese / German',
                                     'European American','Scottish-German','English / Irish', 'French Canadian',
                                     'French Canadian/American','Irish/Spanish','Irish / Mexican', 'Appalachian / American',
                                     'Scotttish/Irish/Norwegian','Chinese/German','German / American',
                                     'Irish/Italian/English/Dutch','Japanese / Scandinavian','Irish/Italian',
                                     'Spanish/Italian','Italian / French', 'British / American','European / American',
                                     'Italian / Portuguese / American', 'Mexican/Greek', 'Irish American',
                                     'African / European / Carribean','Irish / American', 'Lebanese French Canadian',
                                     'Korean / English','Western European / American'], 8)
    
    # Other: 9 
    df['Race'] = df['Race'].replace(['Viking','Other','Elf','Human','other','Vikng','Pennsylvania Dutch','Texan','Yankee',
                                     'Hawaiian','Melungeon','American','Angelic','Alaska Native','Jewish',
                                     'American Mutt','Appalachian American', 'Polsih American'], 9)
    
    
     

clean_9(at)

In [8]:
# I break this column up into North Americans, Europeans, Africans, Australians 
def clean_10(df):
# North Americans 
    df['Country'] = df['Country'].replace(['USA','Canada','CANADA'],0)
#Europeans
    df['Country'] = df['Country'].replace(['England','Germany','France','HOLLAND','Austria',
                                           'GERMANY','THE NETHERLANDS','Switzerland','SWITZERLAND',
                                           'Ireland','Norway','ITALY','Finland','Sweden','Czech Republic'],1)
# Oceania
    df['Country'] = df['Country'].replace(['Australia','AUSTRALIA','New Zealand'],2)

# African 
    df['Country'] = df['Country'].replace(['South Africa','Morocco (North Africa)'],3)

# Other 
    df['Country'] = df['Country'].replace(['GUAM','Bahamas','India'],4)
    

clean_10(at)
    

In [9]:
def clean_11(df):
    df['Hike_Type'] = df['Hike_Type'].map({'NOBO':0, 'SOBO':1, 'Flip':2 })
clean_11(at)

In [10]:
def clean_12(df):
    df.reset_index(inplace = True, drop = True)
clean_12(at)

In [11]:
at.head()

,Finish,Age,Race,State,Sex,Country,Hike_Type,Start,completion_yr,HikedWDog,WinterHike,Under2KSpent,SchoolCredit,Minority,Vegetarian,DehydrateOwnFood,Giardia,Lyme,Norovirus,Total_Days
0,1948-12-31,29.0,7,PA,0,0,0,1948-01-01,1948,False,False,False,False,False,False,False,False,False,False,365 days
1,1951-10-10,28.0,7,CT,0,0,1,1951-05-30,1951,False,False,False,False,False,False,False,False,False,False,133 days
2,1951-09-30,29.0,7,GA,0,0,0,1951-05-31,1951,False,False,False,False,False,False,False,False,False,False,122 days
3,1959-09-17,20.0,0,GA,0,0,0,1959-05-23,1959,False,False,False,False,False,False,False,False,False,False,117 days
4,1965-09-23,29.0,7,PA,0,0,1,1965-04-16,1965,False,False,False,False,False,False,False,False,False,False,160 days


In [19]:
from collections import Counter
Counter(at['State'])

Counter({'AB': 9,
         'AE': 3,
         'AK': 48,
         'AL': 149,
         'AP': 1,
         'AR': 39,
         'AZ': 74,
         'Au': 3,
         'BC': 13,
         'BY': 1,
         'CA': 328,
         'CH': 2,
         'CO': 272,
         'CT': 396,
         'Ca': 4,
         'Co': 2,
         'Cu': 2,
         'Cz': 2,
         'DC': 43,
         'DE': 48,
         'En': 8,
         'FL': 667,
         'Fi': 1,
         'Fr': 2,
         'GA': 834,
         'Ga': 1,
         'Ge': 2,
         'Gu': 1,
         'HI': 21,
         'Ha': 2,
         'Ho': 1,
         'IA': 64,
         'ID': 26,
         'IL': 239,
         'IN': 225,
         'Il': 1,
         'Ir': 1,
         'KS': 52,
         'KY': 168,
         'Ke': 2,
         'LA': 72,
         'La': 3,
         'MA': 738,
         'MB': 1,
         'MD': 352,
         'ME': 522,
         'MI': 358,
         'MN': 155,
         'MO': 142,
         'MS': 62,
         'MT': 39,
         'MY': 1,
         'Ma': 2,
   

In [12]:
features = [value for value in at.columns if value != 'Total_Days']
X = at[features]
y = at['Total_Days']

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2003)

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
lm = LinearRegression()

In [17]:
lm.fit(X_train,y_train)

TypeError: float() argument must be a string or a number, not 'Timestamp'

In [ ]:
pred = lm.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,pred)

In [ ]:
#plt.figure(figsize = (10,10))
#null.plot(kind = 'bar')

In [ ]:
# Exploratory analysis

In [ ]:
# Null values by column 
#plt.figure(figsize = (10,10))
#z = at.isnull().sum()
#z.plot(kind = 'bar')